<a href="https://colab.research.google.com/github/darshlukkad/Unsloth/blob/main/grpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Install core deps (Colab) ---
%pip -q install -U unsloth trl transformers datasets accelerate peft bitsandbytes einops evaluate sentencepiece

# --- Stability flags BEFORE importing transformers/trl/peft ---
import os, sys, platform, torch
os.environ["UNSLOTH_COMPILE_DISABLE"] = "1"   # avoid flaky compiled kernels on some Colab builds
os.environ["UNSLOTH_STABLE_DOWNLOADS"] = "1"  # quieter, more robust HF downloads

# Import Unsloth FIRST so it can patch transformers properly
import unsloth
from unsloth import FastLanguageModel, is_bfloat16_supported

# Now import GRPO trainer/config from TRL
from trl import GRPOTrainer, GRPOConfig
from datasets import load_dataset

# Basic env printouts
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
!nvidia-smi || echo "No NVIDIA GPU detected"

# Precision & common constants
dtype = torch.bfloat16 if is_bfloat16_supported() else torch.float16
device = "cuda" if torch.cuda.is_available() else "cpu"
MAX_SEQ_LEN = 2048
MAX_PROMPT  = 512
MAX_COMP    = 256
print("dtype:", dtype, "| device:", device)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 238.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.3/351.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curren

In [2]:
# --- Install core deps (Colab) ---
%pip -q install -U unsloth trl transformers datasets accelerate peft bitsandbytes einops evaluate sentencepiece

# --- Stability flags BEFORE importing transformers/trl/peft ---
import os, sys, platform, torch
os.environ["UNSLOTH_COMPILE_DISABLE"] = "1"   # avoid flaky compiled kernels on some Colab builds
os.environ["UNSLOTH_STABLE_DOWNLOADS"] = "1"  # quieter, more robust HF downloads

# Import Unsloth FIRST so it can patch transformers properly
import unsloth
from unsloth import FastLanguageModel, is_bfloat16_supported

# Now import GRPO trainer/config from TRL
from trl import GRPOTrainer, GRPOConfig
from datasets import load_dataset

# Basic env printouts
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
!nvidia-smi || echo "No NVIDIA GPU detected"

# Precision & common constants
dtype = torch.bfloat16 if is_bfloat16_supported() else torch.float16
device = "cuda" if torch.cuda.is_available() else "cpu"
MAX_SEQ_LEN = 2048
MAX_PROMPT  = 512
MAX_COMP    = 256
print("dtype:", dtype, "| device:", device)


Python: 3.12.12
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Torch: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
Mon Nov 10 05:20:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P0             25W /   70W |     102MiB /  15360MiB |      0%      Default |
|                      

In [3]:
# Policy model: 4-bit (QLoRA style) + LoRA adapters
import torch
from unsloth import FastLanguageModel

model_id = "HuggingFaceTB/SmolLM2-135M"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name      = model_id,
    max_seq_length  = MAX_SEQ_LEN,   # from Cell 1
    dtype           = torch.bfloat16 if (hasattr(torch.cuda, "is_bf16_supported") and torch.cuda.is_bf16_supported()) else torch.float16,
    load_in_4bit    = True,          # quantized base for low VRAM
)

# Attach LoRA adapters (trainable portion of the policy)
model = FastLanguageModel.get_peft_model(
    model,
    r                          = 16,
    lora_alpha                 = 16,
    lora_dropout               = 0.05,
    target_modules             = ["q_proj","k_proj","v_proj","o_proj",
                                  "gate_proj","up_proj","down_proj"],
    use_gradient_checkpointing = "unsloth",
    random_state               = 3407,
    max_seq_length             = MAX_SEQ_LEN,
)

# Tokenizer safety defaults
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("Loaded policy base:", model_id)
print("LoRA: r=16, alpha=16, dropout=0.05")
print("Device:", model.device, "| 4-bit:", True)


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

HuggingFaceTB/SmolLM2-135M does not have a padding token! Will use pad_token = <|endoftext|>.


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.2 patched 30 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Loaded policy base: HuggingFaceTB/SmolLM2-135M
LoRA: r=16, alpha=16, dropout=0.05
Device: cuda:0 | 4-bit: True


In [4]:
# Load a small slice of GSM8K and build reasoning prompts with gold answers
from datasets import load_dataset
import re

# GSM8K answers usually end with a line like "#### 42"
FINAL_NUM_RE = re.compile(r"####\s*(-?\d+(?:\.\d+)?)")

def parse_gold(s: str):
    if not isinstance(s, str):
        return None
    m = FINAL_NUM_RE.search(s)
    return m.group(1) if m else None

# Pull a manageable subset for the demo
raw = load_dataset("openai/gsm8k", "main", split="train[:1000]")

def build_example(ex):
    q = ex["question"].strip()
    a = ex["answer"].strip()
    gold = parse_gold(a)
    # Encourage chain-of-thought style + explicit final <answer> tag
    system = "You are a careful math tutor. Think step by step and show your work."
    prompt = (
        f"{system}\n\n"
        f"<reasoning>\nSolve the following word problem carefully:\n{q}\n</reasoning>\n"
        f"<answer>\n"
    )
    return {"prompt": prompt, "gold": a, "gold_value": gold}

ds = raw.map(build_example, remove_columns=raw.column_names)
print(ds[0])

# Keep a small working slice for quick GRPO runs; increase later for quality
WORK_SAMPLES = 200
train_ds = ds.select(range(min(WORK_SAMPLES, len(ds))))
print("Train examples:", len(train_ds))
print("Sample prompt:\n", train_ds[0]["prompt"][:400], "...\n")
print("Gold (truncated):\n", train_ds[0]["gold"][:300], "\nGold value:", train_ds[0]["gold_value"])


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'prompt': 'You are a careful math tutor. Think step by step and show your work.\n\n<reasoning>\nSolve the following word problem carefully:\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\n</reasoning>\n<answer>\n', 'gold': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72', 'gold_value': '72'}
Train examples: 200
Sample prompt:
 You are a careful math tutor. Think step by step and show your work.

<reasoning>
Solve the following word problem carefully:
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
</reasoning>
<answer>
 ...

Gold (truncated):
 Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72 
Gold value: 72


In [5]:
# Reward functions for GRPO
# - correctness_reward: compares the model's final numeric answer to the gold value
# - format_reward: encourages closing </answer> tag (and optional reasoning section)

import re

# We look for the last number in the assistant's completion. Adjust as needed.
FINAL_NUM_RE = re.compile(r"(-?\d+(?:\.\d+)?)")

def _extract_numeric(s: str):
    if not isinstance(s, str):
        return None
    s2 = s.replace(",", "")
    m = FINAL_NUM_RE.search(s2)
    return m.group(1) if m else None

def correctness_reward(prompts, completions, metadatas=None, **kwargs):
    """
    Args (as provided by TRL GRPO):
      prompts:     list[str]
      completions: list[list[{'role':'assistant','content': '...'}]]  # n_generations per prompt
      metadatas:   list[dict] with fields we placed into the dataset (e.g., 'gold_value')
    Returns:
      list[float] of rewards, one per completion (flattened 1:1 with 'completions')
    """
    rewards = []
    # If completions are grouped per prompt, flatten while aligning with metadatas
    # TRL passes completions as list of lists; we'll iterate outer then inner
    idx = 0
    for i, gen_group in enumerate(completions):
        gold_v = None
        if metadatas and i < len(metadatas):
            gold_v = str(metadatas[i].get("gold_value", "") if metadatas[i] else "")
        for gen in gen_group:
            text = gen.get("content", "") if isinstance(gen, dict) else str(gen)
            pred_v = _extract_numeric(text)
            if pred_v is not None and gold_v is not None and pred_v == gold_v:
                rewards.append(2.0)  # correct answer bonus
            else:
                rewards.append(0.0)
            idx += 1
    return rewards

def format_reward(prompts, completions, **kwargs):
    """
    Small structural bonus to stabilize outputs:
      +1 if '</answer>' appears,
      +0.5 if '<reasoning>' also appears.
    """
    rewards = []
    for gen_group in completions:
        for gen in gen_group:
            text = gen.get("content", "") if isinstance(gen, dict) else str(gen)
            r = 0.0
            if "</answer>" in text:
                r += 1.0
            if "<reasoning>" in text:
                r += 0.5
            rewards.append(r)
    return rewards

REWARD_FUNCS = [correctness_reward, format_reward]
print("Defined reward functions:", [f.__name__ for f in REWARD_FUNCS])


Defined reward functions: ['correctness_reward', 'format_reward']


In [10]:
# Robust rewards: handle completion_ids_list OR completions, grouped or flat, list or tensor.
import re, torch
from trl import GRPOTrainer, GRPOConfig

FINAL_NUM_RE = re.compile(r"(-?\d+(?:\.\d+)?)")

def _extract_numeric(s: str):
    if not isinstance(s, str):
        return None
    s2 = s.replace(",", "")
    m = FINAL_NUM_RE.search(s2)
    return m.group(1) if m else None

def _to_groups(obj, num_generations: int):
    """
    Return a list of groups (one group per prompt), each group is a list of
    sequences (either token ids or text). Supports:
      - list[list[...]] (already grouped)
      - list[...] (flat -> chunk by num_generations)
      - torch.Tensor of shape [N, L] (flat) or [B, K, L] (grouped)
    """
    if obj is None:
        return None
    # Tensor cases
    if isinstance(obj, torch.Tensor):
        if obj.dim() == 3:  # [B, K, L]
            B, K, _ = obj.shape
            return [ [ obj[b, k].tolist() for k in range(K) ] for b in range(B) ]
        elif obj.dim() == 2:  # [N, L] flat -> chunk by K
            N, _ = obj.shape
            flat = [ obj[n].tolist() for n in range(N) ]
            return [ flat[i:i+num_generations] for i in range(0, len(flat), num_generations) ]
        else:
            # Unexpected tensor rank; treat as flat list of one group
            return [ [ obj.tolist() ] ]
    # Python list cases
    if isinstance(obj, list):
        if len(obj) > 0 and isinstance(obj[0], list):
            return obj  # already grouped
        else:
            # flat -> chunk by K
            return [ obj[i:i+num_generations] for i in range(0, len(obj), num_generations) ]
    # Fallback: single group
    return [ [ obj ] ]

def _decode_groups(tokenizer, groups, are_ids: bool):
    """Decode groups to text; keep the same 2-level structure."""
    out = []
    for g in groups:
        gg = []
        for item in g:
            if are_ids:
                gg.append(tokenizer.decode(item, skip_special_tokens=True))
            else:
                # item can be dict ({"role":"assistant","content":...}) or text or tensor->str
                if isinstance(item, dict):
                    gg.append(str(item.get("content", "")))
                else:
                    gg.append(str(item))
        out.append(gg)
    return out

def _flatten(groups):
    flat = []
    owners = []
    for i, g in enumerate(groups):
        for x in g:
            flat.append(x)
            owners.append(i)
    return flat, owners

def correctness_reward(prompts, completions=None, completion_ids_list=None, metadatas=None, num_generations=4, **kwargs):
    """One scalar per completion."""
    # Prefer token ids if available; else use text
    groups_ids = _to_groups(completion_ids_list, num_generations) if completion_ids_list is not None else None
    if groups_ids is not None:
        groups_txt = _decode_groups(tokenizer, groups_ids, are_ids=True)
    else:
        groups_raw = _to_groups(completions, num_generations)
        groups_txt = _decode_groups(tokenizer, groups_raw, are_ids=False)

    flat_txt, owners = _flatten(groups_txt)

    # Build gold list aligned to 'owners' (owner = prompt index)
    golds = []
    if metadatas:
        for md in metadatas:
            if md is None:
                golds.append(None)
            else:
                gv = md.get("gold_value", None)
                golds.append(str(gv) if gv is not None else None)

    rewards = []
    for t, owner in zip(flat_txt, owners):
        pred = _extract_numeric(t)
        gold = golds[owner] if owner < len(golds) else None
        rewards.append(2.0 if (pred is not None and gold is not None and pred == gold) else 0.0)

    # Sanity: ensure length matches total completions
    expected = sum(len(g) for g in groups_txt)
    assert len(rewards) == expected, f"correctness_reward produced {len(rewards)} != expected {expected}"
    return rewards

def format_reward(prompts, completions=None, completion_ids_list=None, num_generations=4, **kwargs):
    """Small structural bonus encouraging tags."""
    groups_ids = _to_groups(completion_ids_list, num_generations) if completion_ids_list is not None else None
    if groups_ids is not None:
        groups_txt = _decode_groups(tokenizer, groups_ids, are_ids=True)
    else:
        groups_raw = _to_groups(completions, num_generations)
        groups_txt = _decode_groups(tokenizer, groups_raw, are_ids=False)

    flat_txt, _ = _flatten(groups_txt)
    rewards = []
    for t in flat_txt:
        r = 0.0
        if "</answer>" in t:   r += 1.0
        if "<reasoning>" in t: r += 0.5
        rewards.append(r)

    expected = sum(len(g) for g in groups_txt)
    assert len(rewards) == expected, f"format_reward produced {len(rewards)} != expected {expected}"
    return rewards

REWARD_FUNCS = [correctness_reward, format_reward]
print("Patched v3 reward functions:", [f.__name__ for f in REWARD_FUNCS])

# Ensure metadata present
def add_metadata(ex):
    return {"metadata": {"gold_value": ex["gold_value"]}}
train_grpo = train_ds.map(add_metadata)

# Keep GRPO config; Unsloth will align per_device_train_batch_size to num_generations
cfg = GRPOConfig(
    output_dir="outputs_grpo_smollm2",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    num_train_epochs=1,
    logging_steps=5,
    save_steps=50,
    save_total_limit=2,
    report_to="none",
    fp16=(dtype == torch.float16),
    bf16=(dtype == torch.bfloat16),

    max_prompt_length=MAX_PROMPT,
    max_completion_length=MAX_COMP,
    num_generations=4,
    temperature=0.7,
    top_p=0.95,
)

# Stability
if hasattr(model.config, "use_cache"):
    model.config.use_cache = False

trainer = GRPOTrainer(
    model=model,
    reward_funcs=REWARD_FUNCS,
    args=cfg,
    train_dataset=train_grpo,   # contains 'prompt' and 'metadata'
    tokenizer=tokenizer,
)

train_out = trainer.train()
print("GRPO training complete.")
print(train_out)

# Save ONLY the LoRA adapter (compact)
adapter_dir = "smollm2_grpo_adapter"
trainer.model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(adapter_dir)
print("Saved GRPO LoRA adapter ->", adapter_dir)


Patched v3 reward functions: ['correctness_reward', 'format_reward']


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 4,884,480 of 139,399,488 (3.50% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / correctness_reward / mean,rewards / correctness_reward / std,rewards / format_reward / mean,rewards / format_reward / std
5,0.000000,0.656250,0.592648,256.000000,256.000000,256.000000,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0.000011,0.000000,0.000000,0.656250,0.572258
10,0.000000,0.693750,0.426156,256.000000,256.000000,256.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000011,0.000000,0.000000,0.693750,0.504825
15,0.000000,0.737500,0.497031,256.000000,256.000000,256.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000010,0.000000,0.000000,0.737500,0.517777
20,0.000000,0.618750,0.508958,256.000000,256.000000,256.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000010,0.000000,0.000000,0.618750,0.520622
25,0.000000,0.612500,0.391393,256.000000,256.000000,256.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000012,0.000000,0.000000,0.612500,0.470663
30,0.000000,0.568750,0.464298,256.000000,256.000000,256.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000015,0.000000,0.000000,0.568750,0.492589
35,0.000000,0.706250,0.503638,256.000000,256.000000,256.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000014,0.000000,0.000000,0.706250,0.568450
40,0.000000,0.600000,0.486481,256.000000,256.000000,256.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000014,0.000000,0.000000,0.600000,0.554920
45,0.000000,0.656250,0.502341,256.000000,256.000000,256.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000015,0.000000,0.000000,0.656250,0.565595
50,0.000000,0.718750,0.427898,256.000000,256.000000,256.000000,1.000000,0.000000,0.000000,0.000000,No Log,No Log,No Log,No Log,No Log,0.000015,0.000000,0.000000,0.718750,0.506418


Unsloth: Will smartly offload gradients to save VRAM!
GRPO training complete.
TrainOutput(global_step=50, training_loss=3.024935722351074e-08, metrics={'train_runtime': 1190.8664, 'train_samples_per_second': 0.168, 'train_steps_per_second': 0.042, 'total_flos': 0.0, 'train_loss': 3.024935722351074e-08})
Saved GRPO LoRA adapter -> smollm2_grpo_adapter


In [11]:
# Clean reload for inference: base (4-bit) + GRPO LoRA adapter, then sample a few problems
import os, re, random, torch
from unsloth import FastLanguageModel
from peft import PeftModel

base_id     = "HuggingFaceTB/SmolLM2-135M"
adapter_dir = "smollm2_grpo_adapter"
assert os.path.isdir(adapter_dir), "Adapter folder not found. Run the GRPO training cell first."

dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Reload base in 4-bit for low-VRAM inference
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = base_id,
    max_seq_length = 2048,
    dtype          = dtype,
    load_in_4bit   = True,
)

# Attach LoRA adapter (GRPO policy)
model = PeftModel.from_pretrained(base_model, adapter_dir)
model.eval()

# Tokenizer safety defaults
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

FINAL_NUM_RE = re.compile(r"(-?\d+(?:\.\d+)?)")

def extract_num(s: str):
    if not isinstance(s, str): return None
    s = s.replace(",", "")
    m = FINAL_NUM_RE.search(s)
    return m.group(1) if m else None

def generate(prompt, max_new_tokens=192, temperature=0.7, top_p=0.95):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

# Pick up to 3 random examples from train_ds (built earlier)
N = 3
idxs = random.sample(range(len(train_ds)), k=min(N, len(train_ds)))
for k, i in enumerate(idxs, 1):
    ex = train_ds[i]
    prompt = ex["prompt"]
    gold_text = ex["gold"]
    gold_val  = str(ex.get("gold_value"))

    print(f"\n=== Example {k} ===")
    print("PROMPT:\n", prompt[:400], "...\n")
    print("GOLD (truncated):\n", gold_text[:300], "\nGold value:", gold_val)

    gen = generate(prompt)
    print("\n--- MODEL OUTPUT ---\n", gen)

    # Try to show continuation after <answer> if present
    if "<answer>" in gen:
        cont = gen.split("<answer>", 1)[-1]
        print("\n--- CONTINUATION AFTER <answer> ---\n", cont.strip())

    pred = extract_num(gen)
    print("\nPRED NUM:", pred, "| GOLD NUM:", gold_val, "| CORRECT:", (pred == gold_val))


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
HuggingFaceTB/SmolLM2-135M does not have a padding token! Will use pad_token = <|endoftext|>.

=== Example 1 ===
PROMPT:
 You are a careful math tutor. Think step by step and show your work.

<reasoning>
Solve the following word problem carefully:
Joy can read 8 pages of a book in 20 minutes. How many hours will it take her to read 120 pages?
</reasoning>
<answer>
 ...

GOLD (truncated):
 In one hour, there are 3 sets of 20 minutes.
So, Joy can read 8 x 3 = <<8*3=24>>24 pages in an hour.
It will take her 120/24 = <<120/24=5>>5 hours to r

In [13]:
# Optional: Merge GRPO LoRA weights into a single model and export to GGUF for Ollama
import os, glob, torch
from unsloth import FastLanguageModel

base_id     = "HuggingFaceTB/SmolLM2-135M"
adapter_dir = "smollm2_grpo_adapter"
merged_dir  = "smollm2_grpo_merged_16bit"
gguf_dir    = "gguf_export_grpo_q8"

assert os.path.isdir(adapter_dir), "Adapter folder not found. Run the GRPO training cell first."

# If 'model'/'tokenizer' not in memory (fresh runtime), rebuild them
if "model" not in globals() or "tokenizer" not in globals():
    base_model, tokenizer = FastLanguageModel.from_pretrained(
        model_name     = base_id,
        max_seq_length = 2048,
        dtype          = torch.float16 if torch.cuda.is_available() else torch.float32,
        load_in_4bit   = True,
    )
    from peft import PeftModel
    model = PeftModel.from_pretrained(base_model, adapter_dir)

# 1) Try Unsloth's native merge first
merged_ok = False
if hasattr(model, "save_pretrained_merged"):
    try:
        model.save_pretrained_merged(merged_dir, tokenizer, save_method="merged_16bit")
        print("Merged LoRA →", merged_dir, "(Unsloth merged_16bit).")
        merged_ok = True
    except Exception as e:
        print("Unsloth merged_16bit failed, falling back to PEFT:", e)

# 2) Fallback: PEFT merge_and_unload
if not merged_ok:
    try:
        from peft import PeftModel
        merged = model.merge_and_unload()
        os.makedirs(merged_dir, exist_ok=True)
        merged.save_pretrained(merged_dir)
        tokenizer.save_pretrained(merged_dir)
        print("Merged LoRA →", merged_dir, "(PEFT fallback).")
        merged_ok = True
    except Exception as e:
        print("PEFT merge fallback failed:", e)

# 3) Export to GGUF for Ollama (Q8_0 quant)
if merged_ok:
    os.makedirs(gguf_dir, exist_ok=True)
    try:
        merged_model, merged_tok = FastLanguageModel.from_pretrained(
            model_name      = merged_dir,
            max_seq_length  = 2048,
            dtype           = torch.float16 if torch.cuda.is_available() else torch.float32,
            load_in_4bit    = False,
            full_finetuning = False,
        )

        merged_model.save_pretrained_gguf(
            gguf_dir,
            merged_tok,
            quantization_method="q8_0",
        )
        print("Saved GGUF to:", gguf_dir)

        # Write a simple Modelfile for Ollama
        ggufs = glob.glob(os.path.join(gguf_dir, "*.gguf"))
        gguf_name = os.path.basename(ggufs[0]) if ggufs else "model-Q8_0.gguf"
        with open(os.path.join(gguf_dir, "Modelfile"), "w") as f:
            f.write(f"FROM ./{gguf_name}\n")
            f.write("PARAMETER temperature 0.7\n")
            f.write("PARAMETER top_p 0.9\n")
            f.write('TEMPLATE "{{ .System }}\\n\\n{{ .Prompt }}"\\n')
        print("Wrote Modelfile →", os.path.join(gguf_dir, "Modelfile"))

        print("\nNext steps (on your machine):")
        print(f"cd {gguf_dir}")
        print("ollama create smollm2-grpo -f Modelfile")
        print("ollama run smollm2-grpo")
    except Exception as e:
        print("GGUF export skipped or failed:", e)
else:
    print("Merge did not succeed; skipping GGUF export.")


/usr/local/lib/python3.12/dist-packages/unsloth_zoo/saving_utils.py:969: UserWarning: Model is not a PeftModel (no Lora adapters detected). Skipping Merge. Please use save_pretrained() or push_to_hub() instead!
  warnings.warn("Model is not a PeftModel (no Lora adapters detected). Skipping Merge. Please use save_pretrained() or push_to_hub() instead!")


Merged LoRA → smollm2_grpo_merged_16bit (Unsloth merged_16bit).
GGUF export skipped or failed: Unsloth: No config file found - are you sure the `model_name` is correct?
If you're using a model on your local device, confirm if the folder location exists.
If you're using a HuggingFace online model, check if it exists.
